This notebook parses the benchmark results.

In [14]:
from pathlib import Path
from typing import NamedTuple
from itertools import product

import pandas as pd

## Constants

In [15]:
DATA_DIR = Path("../data/results/")
PROBLEMS = [
    "JSP",
    "FJSP",
    "NPFSP",
    "NW-PFSP",
    "HFSP",
    "PFSP",
    "SDST-PFSP",
    "TCT-PFSP",
    "TT-PFSP",
    "RCPSP",
    "MMRCPSP",
    "RCMPSP"
]
SOLVERS = ["ortools", "cpoptimizer"]
TIME_LIMITS = [120, 900]

## Helpers

In [16]:
class Result(NamedTuple):
    instance: str
    status: str
    objective: float
    time: float

def parse_results(loc: Path) -> list[Result]:
    with open(loc, "r") as fh:
        text = fh.read()

    text = text[text.index("Instance"):] # start at results header
    lines = [line.strip() for line in text.split("\n")]
    lines = lines[2:] # skip header
    lines = iter(lines)

    results = []
    while (line := next(lines)): # continue until next empty line
        instance, status, objective, time = line.split()
        result = Result(instance, status, float(objective), float(time))
        results.append(result)

    return results

## Parsing

In [17]:
class Experiment(NamedTuple):
    problem: str
    solver: str
    time_limit: int
    instance: str
    status: str
    objective: float
    time: float

data = [] 
for problem, solver, time_limit in product(PROBLEMS, SOLVERS, TIME_LIMITS):
    try:
        loc = DATA_DIR / problem / solver / str(time_limit) / "results.txt"
        results = parse_results(loc)
        
        for result in results:
            data.append(Experiment(problem, solver, time_limit, *result))
    except:
        print(loc)

In [18]:
df = pd.DataFrame(data)
df.head()

,problem,solver,time_limit,instance,status,objective,time
0,JSP,ortools,120,1.txt,Optimal,1231.0,4.30
1,JSP,ortools,120,10.txt,Optimal,1241.0,15.37
2,JSP,ortools,120,100.txt,Feasible,3871.0,120.02
3,JSP,ortools,120,101.txt,Feasible,4459.0,120.06
4,JSP,ortools,120,102.txt,Optimal,4725.0,98.81


In [19]:
df.to_csv("../data/results.csv", index=False)

## Extra results

Extra results by running 16 cores instead of 8.

In [20]:
DATA_DIR = Path("../data/extra/core16")
PROBLEMS = [
    "JSP",
    "FJSP",
    "NPFSP",
    "NW-PFSP",
    "HFSP",
]
SOLVERS = ["ortools", "cpoptimizer"]
TIME_LIMITS = [900]

In [21]:
data = [] 
for problem, solver, time_limit in product(PROBLEMS, SOLVERS, TIME_LIMITS):
    try:
        loc = DATA_DIR / problem / solver / str(time_limit) / "results.txt"
        results = parse_results(loc)
        
        for result in results:
            data.append(Experiment(problem, solver, time_limit, *result))
    except Exception as e:
        print(loc, e)

In [22]:
df = pd.DataFrame(data)
df.head()

,problem,solver,time_limit,instance,status,objective,time
0,JSP,ortools,900,1.txt,Optimal,1231.0,4.75
1,JSP,ortools,900,10.txt,Optimal,1241.0,47.35
2,JSP,ortools,900,100.txt,Feasible,3786.0,900.18
3,JSP,ortools,900,101.txt,Optimal,4380.0,330.02
4,JSP,ortools,900,102.txt,Optimal,4725.0,111.31


In [24]:
df.to_csv("../data/cores16.csv", index=False)